In [ ]:
pip install hazm

In [ ]:
from __future__ import print_function, unicode_literals
import codecs, subprocess, random
from collections import Counter
from itertools import islice
from nltk.tag import untag
from sklearn.model_selection import train_test_split
from hazm import *
from hazm.Chunker import tree2brackets
from hazm.PeykareReader import coarse_pos_e as peykare_coarse_pos_e
from hazm.DadeganReader import coarse_pos_e as dadegan_coarse_pos_e
import pandas as pd
from pandas_profiling import ProfileReport
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
# from google.cloud import vision
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import nltk
import glob
import re
import os
import io

# define **stopwords**

In [ ]:
#stop word
stop_words = pd.read_json('/content/drive/MyDrive/Colab Notebooks/emotion_project/stopwords-fa.json', encoding= 'utf-8')
stopwords=stop_words.transpose()
stop_words=stopwords.values.tolist()
stopwords[1]='!!'
stop_words=stopwords.values.tolist()
ss=stopwords.iloc[0,9:798]
stop_wordss=ss.values.tolist()

# **clean data function**

In [ ]:
# clean data function
!pip install emojis
import emojis
import re
import string
from string import digits
def text_cleaner(text):

  # Emoji parsing
  text = emojis.decode(str(text))
  
  # Removing numbers
  text = re.sub(r'\\d+', '', text)

  # Removing single character words
  text =  re.sub(r"\b[a-zA-Z]\b", "", text)
  # Removing multiple spaces
  text = re.sub(r'\s+', ' ', text)

  # Slang parsing
  # for slang, meaning in slangs.items():
  #   text = text.replace(slang, meaning)
  
  text=text.replace(":", "")
  text=text.replace('@', "")
  text=text.replace("#", " ")
  text=text.replace("_", " ")
  text=text.replace("،", " ")
  text=text.replace("\u200c", " ")
  text =  re.sub(r"[a-zA-Z]", "", text)
  
  # Removing links
  text=text.replace(r"http\S+", "")
  text=text.translate(str.maketrans('', '', string.punctuation))
  text=text.translate(str.maketrans('', '', string.digits))
  # text = text.translate(None, digits)

  # Normalizing Case
  # Filter Out Punctuation
  # Filter out Stop Words (and Pipeline)

  # words = [w for w in text.split() if w.isalpha() and not w in sss]
  
  words = [w for w in text.split()if not w in stop_wordss]
  return ' '.join(words)

# **clean data and stemming**

In [ ]:
#clean data and stemming
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/Emotion.csv', encoding= 'utf-8')
df.loc[0,'text']
clean_text=[]
stemmer = Stemmer()
for i in range(len(df)) :
  text=text_cleaner(df.loc[i,'text'])
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text.append(t)
type(clean_text)
df.head()

,text,label
0,قبلا میگفتن اگه درس نخونی معلم میزندت، الان در...,0. 0. 0. 0. 0. 0. 0. 0. 0. 1.\n
1,یه شات هست از کلاب‌هاوس که هی توییت و لایک و ک...,0. 0. 0. 0. 0. 0. 0. 0. 0. 1.\n
2,ایران انقده خوبه که اصن همه میدونن منو بفرستین...,1. 0. 0. 0. 0. 0. 0. 0. 0. 0.\n
3,رد پای من هر جا بری هس.,0. 0. 0. 1. 0. 0. 0. 0. 0. 0.\n
4,#علی_جوانمردیآقای #علی‌جوانمردی؛در این‌که شما ...,0. 1. 0. 0. 1. 0. 0. 0. 0. 0.\n


# **tf idf vectorize**

In [ ]:
#tf idf vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features= 700)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(clean_text).toarray()
feat_names =tfidf_vectorizer.get_feature_names()
features = pd.DataFrame(tfidf_vectorizer_vectors, columns= feat_names)
tfidf_vectorizer_vectors.shape


(5999, 700)

# **test data processing**

In [ ]:
#test data
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/EmotionTest.csv', encoding= 'utf-8')
clean_text_test=[]
stemmer = Stemmer()
for i in range(len(df_test)) :
  text=text_cleaner(df_test.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text_test.append(t)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(clean_text_test).toarray()
feat_names =tfidf_vectorizer.get_feature_names()
features_test = pd.DataFrame(tfidf_vectorizer_vectors, columns= feat_names)
df_clean2=pd.DataFrame(clean_text_test)
df_clean2.columns=['text']
text_feat='text'
related_word_features_extraction(df_clean2, text_feat)
df_featurs2 = pd.concat([df_clean2, features_test], axis=1)
df_featurs2.drop(text_feat, axis=1, inplace=True)
y_test=df_test.loc[:,'label']
y_test.replace('\n','')
y_test=y_test.values.tolist()
x_test=df_featurs2.values.tolist()
for i in range(len(y_test)):
  y_test[i]=y_test[i].replace('\n','')
  y_test[i]=y_test[i].replace('.','')

ytest=[]
for i in range(len(y_test)):
  yy = [int(t) for t in y_test[i].split(' ')]
  ytest.append(yy)
ytest=np.array(ytest)
x_test=np.array(x_test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **create lexicon from** **NRC**

In [ ]:
#create lexicon from NRC
from pandas import DataFrame
nrc = pd.read_csv('NRC-emotion-lexicon-wordlevel-persian-v0.92.txt', delimiter = "\t")
nrc.columns=['word','emotion','label']
nrc.head()
len(nrc)

for i in range(len(nrc)):
  if int(nrc.loc[i,'label']) :
    with open(nrc.loc[i,'emotion']+'.txt', 'a') as f:
      f.write(nrc.loc[i,'word'])
      f.write('\n')
      

    




In [ ]:

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **create lexicon from related word**

In [ ]:
#create lexicon from related word
def createLexicons (directory,stemDir):

  lexicon = {}

  for name in glob.glob(directory +'*.txt'):
    type_ = re.sub(directory,"",name.split('.')[0])
    content = open(name).read()
    tokens = content.split('\n')
    lexicon[type_] = tokens

    # with open(stemDir + type_ + '.pkl', 'wb') as f:
    #   pickle.dump(tokens, f) 

  return lexicon

In [ ]:
lex_dir = '/content/drive/MyDrive/Colab Notebooks/emotion_project/lex/'

In [ ]:
if not os.path.exists(lex_dir):
    os.makedirs(lex_dir)

lexicon = createLexicons (lex_dir,lex_dir)

with open(lex_dir+'lexicon.pkl', 'wb') as f:
    pickle.dump(lex_dir, f)

print(lexicon)

{'anger': ['خشم ', 'خشمگین', 'تجاوز', 'مهاجم ', 'الکل', 'حمله', 'قساوت', 'بیرحم', 'بی رحم', 'انتقامجو', 'انتقام جو', 'وحشیانه', 'تلخی', 'انفجار', 'محاصره', 'خونخوار', 'خونریزی', 'مجادله ', 'جنگ ', 'دعوا', 'کشتن', 'قتل', 'قاتل', 'مسلح', 'ترور', 'وحشیگری', 'حکم', 'محکومیت', 'جنایتکاری', 'ظالم', 'ظالمانه', 'ظلم', 'سگ', 'سگی', 'تنغیض', 'عصبانی', 'بی اعصاب', 'سرخورده', 'سرخوردگی', 'محروم', 'اواره', 'دیوانه', 'کشتار', 'تظاهرات', 'جنبش', 'اعتراض', 'نظام', 'حکومت', 'مرگ', 'کشنده', 'هیولا', 'ستمگر', 'دردناک', 'غارت', 'سیاست', 'اراذل', 'قصاص ', 'شورش', 'کودتا', 'وحشی', 'فتنه', 'تیراندازی', 'جیغ', 'فریاد', 'گه', 'خفه', 'کثیف', 'اعتصاب', 'خودکشی', 'عصبانیت', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'love': ['عشق', 'معشوقه', 'دلبر', 'دوستی ', 'عاشقانه', 'محبوب', 'محبت', 'عزیز', 'شیفتگی', 'عزیزترین', 'ب

# adding features from lexicon

In [ ]:
#add features from lexicon
lex_dir2 = '/content/drive/MyDrive/Colab Notebooks/emotion_project/lex/lexicon.pkl'

with open(lex_dir2, 'rb') as f:
  relatedWords= pickle.load(f)
  print(relatedWords)

relatedWords

# all_dict = {**relatedWords}

def related_word_features_extraction(df, text_feat):
  for key in lexicon:
    words = set(lexicon[key])  
    df[key] = df[text_feat].apply(lambda t: len(set(t.split()).intersection(words)))

/content/drive/MyDrive/Colab Notebooks/emotion_project/lex/


In [ ]:
df_clean=pd.DataFrame(clean_text)
df_clean.columns=['text']
df_clean.tail()
text_feat='text'
related_word_features_extraction(df_clean, text_feat)
df_clean.tail()

,text,anger,love,disgust,fear,happy,suprise,sadness
5994,یادداشتنگاه وعده انتخابات اردوغ حزب عدال توسعه...,0,0,0,0,0,0,0
5995,رید بابا دفعه قبل گاز خونمون سیگار روشن,0,0,0,0,0,0,0
5996,عزیز ایفون دایرک,0,1,0,0,0,0,0
5997,دیگه عمه ناپلئون احمد نژاد آد میفرستاد برا مذا...,0,0,1,0,0,0,0
5998,چهر ه مشخص بشه مریض جنس درمانگاه مراکز مشاوره ...,0,0,0,0,0,0,0


In [ ]:
df_featurs = pd.concat([df_clean, features], axis=1)
# df_featurs=features
df_featurs.drop(text_feat, axis=1, inplace=True)
df_featurs.head()


,anger,love,disgust,fear,happy,suprise,sadness,آب,آخر,آخه,آخوند,آد,آدم,آدما,آر,آرزو,آزاد,آزار,آشنا,آقا,آمریکا,آموز,آهنگ,آینده,اب,اتاق,اتفاق,اجازه,اجتماع,اجرا,احترا,احساس,احمق,اخبار,اخلاق,اخه,اد,ادامه,ادب,اذ,...,کارا,کامیوندار,کتاب,کثیف,کدو,کر,کره,کسا,کشور,کف,کل,کلاب,کلمه,کله,کمک,کنن,کنه,کنین,کوتاه,کودک,کون,کیر,گذا,گر,گرام,گروه,گریه,گزار,گل,گو,گوش,گوه,گیر,یاد,یار,یهو,یک,۱۲,۲۰۳۰,۲۵
0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.284493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375961,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.229496,0.0,0.174418,0.0,0.0,0.213268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y=df.loc[:,'label']
y.replace('\n','')
y=y.values.tolist()
x=df_featurs.values.tolist()
for i in range(len(y)):
  y[i]=y[i].replace('\n','')
  y[i]=y[i].replace('.','')
label=[]
for i in range(len(y)):
  yy = [int(t) for t in y[i].split(' ')]
  label.append(yy)
label=np.array(label)
x=np.array(x)

# classify

In [ ]:
#classify


# calassifier1

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier())

clf = Pipeline([('classify', classifier)])

# print (clf)
clf.fit(x, label)
print(clf.score(x, label))

0.10085014169028171


In [ ]:
yhat = clf.predict(x_test)

auc_y1 = roc_auc_score(ytest[:,0],yhat[:,0])
auc_y2 = roc_auc_score(ytest[:,1],yhat[:,1])
auc_y3 = roc_auc_score(ytest[:,2],yhat[:,2])
auc_y4 = roc_auc_score(ytest[:,3],yhat[:,3])
auc_y5 = roc_auc_score(ytest[:,4],yhat[:,4])
auc_y6 = roc_auc_score(ytest[:,5],yhat[:,5])
auc_y7 = roc_auc_score(ytest[:,6],yhat[:,6])
auc_y8 = roc_auc_score(ytest[:,7],yhat[:,7])
auc_y9 = roc_auc_score(ytest[:,8],yhat[:,8])
auc_y10 = roc_auc_score(ytest[:,9],yhat[:,9])
print("ROC AUC y1: %.4f, y2: %.4f, y3: %.4f, y4: %.4f, y5: %.4f ,y6: %.4f, y7: %.4f, y8: %.4f, y9: %.4f, y10: %.4f" % (auc_y1, auc_y2, auc_y3, auc_y4, auc_y5,auc_y6, auc_y7, auc_y8, auc_y9, auc_y10))


ROC AUC y1: 0.5010, y2: 0.5039, y3: 0.4990, y4: 0.5000, y5: 0.4998 ,y6: 0.4990, y7: 0.5171, y8: 0.5000, y9: 0.5000, y10: 0.5018


# **classifier2**

In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 3.2 MB/s 


In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score
mlknn_classifier = MLkNN()
mlknn_classifier.fit(x, label)
yhat = mlknn_classifier.predict(x_test)
  
# print(accuracy_score(ytest, predicted))
print(hamming_loss(ytest, yhat))

0.1257


In [ ]:
print(accuracy_score(ytest, yhat))

0.031


In [ ]:
yhat=yhat.toarray()
auc_y1 = roc_auc_score(ytest[:,0],yhat[:,0])
auc_y2 = roc_auc_score(ytest[:,1],yhat[:,1])
auc_y3 = roc_auc_score(ytest[:,2],yhat[:,2])
auc_y4 = roc_auc_score(ytest[:,3],yhat[:,3])
auc_y5 = roc_auc_score(ytest[:,4],yhat[:,4])
auc_y6 = roc_auc_score(ytest[:,5],yhat[:,5])
auc_y7 = roc_auc_score(ytest[:,6],yhat[:,6])
auc_y8 = roc_auc_score(ytest[:,7],yhat[:,7])
auc_y9 = roc_auc_score(ytest[:,8],yhat[:,8])
auc_y10 = roc_auc_score(ytest[:,9],yhat[:,9])
print("ROC AUC y1: %.4f, y2: %.4f, y3: %.4f, y4: %.4f, y5: %.4f ,y6: %.4f, y7: %.4f, y8: %.4f, y9: %.4f, y10: %.4f" % (auc_y1, auc_y2, auc_y3, auc_y4, auc_y5,auc_y6, auc_y7, auc_y8, auc_y9, auc_y10))

ROC AUC y1: 0.5176, y2: 0.5204, y3: 0.5181, y4: 0.5068, y5: 0.5298 ,y6: 0.4995, y7: 0.5382, y8: 0.5000, y9: 0.5117, y10: 0.5626
